In [1]:
import os
import ccxt
import datetime
from configparser import ConfigParser
from binance.client import Client
from utils.functions_bollinger import *
from utils.helpers import *
from config import *

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')
binance_client = Client(api_key=api_key, api_secret=api_secret)

In [3]:
# # Initialize Variables
# holding_quantity = 0
# decimals_quantity = check_decimals(symbol=TRADING_TICKER_NAME, client=binance_client)
# exchange = ccxt.binance()
# currently_holding = False


# ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)

# bollinger_result = 'WAIT'
# ticker_df = ticker_data
# # 20-window moving average 
# ticker_df['sma_bb'] = ticker_df['close'].rolling(window=20).mean()
# # upper and lower bollinger bands: SMA +/- 2 * standard deviation
# ticker_df['stddev'] = ticker_df['close'].rolling(window=20).std()
# ticker_df['upper_boll1'] = ticker_df['sma_bb'] + (1 * ticker_df['stddev'])
# weight=1
# ticker_df['weighted_upper_boll1'] = ticker_df['sma_bb'] + (1 * weight * ticker_df['stddev']) 

* It is recommend to initialize the bot close to the moment that a candle is closed

In [3]:
# Initialize Variables
holding_quantity = 0
decimals_quantity = check_decimals(symbol=TRADING_TICKER_NAME, client=binance_client)
exchange = ccxt.binance()
currently_holding = False

while 1:
    # STEP 1: FETCH THE CANDLE DATA
    ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
    
    if ticker_data is not None:
        # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
        #trade_rec_type = get_trade_recommendation(ticker_data, RSI_OVERSOLD, RSI_OVERBOUGHT)
        trade_rec_type = get_trade_recommendation_MACD(ticker_data)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} {CCXT_TICKER_NAME} TRADING RECOMMENDATION: {trade_rec_type}')

        # STEP 3: EXECUTE THE BUY OPERATION
        if (trade_rec_type == 'BUY' and not currently_holding):
            trade_successful, holding_quantity, order_price = execute_trade(binance_client, trade_rec_type, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
            currently_holding = not currently_holding if trade_successful else currently_holding
            buy_position = True
            
            # CHECK SELL OPERATION
            while buy_position:
                ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
                confirm_sell = confirm_sell_operation(ticker_data, 1, exchange, CCXT_TICKER_NAME, order_price)

                if (confirm_sell == "SELL"):
                    trade_successful, holding_quantity, order_price = execute_trade(binance_client, confirm_sell, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
                    currently_holding = not currently_holding if trade_successful else currently_holding
                    buy_position = not buy_position
                
                time.sleep(15)

        time.sleep(CANDLE_DURATION_IN_MIN*60)
    else:
        print(f'Unable to fetch ticker data - {CCXT_TICKER_NAME}. Retrying!!')

        time.sleep(5)

22/03/2023 00:00:06 DOGE/USDT TRADING RECOMMENDATION: WAIT
22/03/2023 00:15:06 DOGE/USDT TRADING RECOMMENDATION: WAIT
22/03/2023 00:30:07 DOGE/USDT TRADING RECOMMENDATION: BUY
PLACING ORDER 22/03/2023 00:30:08: DOGEUSDT, BUY, 0.07632, 144.0, 1679455808259 
ORDER PLACED
Waiting operation filling...
ORDER EXECUTED!
22/03/2023 00:30:19: Last price: 0.07628; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss Limit: 0.0759384
22/03/2023 00:30:35: Last price: 0.07624; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss Limit: 0.0759384
22/03/2023 00:30:50: Last price: 0.07621; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss Limit: 0.0759384
22/03/2023 00:31:06: Last price: 0.07617; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss Limit: 0.0759384
22/03/2023 00:31:22: Last price: 0.07619; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss Limit: 0.0759384
22/03/2023 00:31:37: Last price: 0.07612; Weighted Bollinger 1 Sigma:0.0764074451360813; Price Loss 

In [5]:
# CCXT_TICKER_NAME = ['LUNA/USDT', 'DOT/USDT', 'DOGE/USDT', 'AVAX/USDT', 'MATIC/USDT']
# TRADING_TICKER_NAME = ['LUNAUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'MATICUSDT']

# currently_holding = False
# while 1:
#     for ccxt_ticker_name, trading_ticker_name in zip(CCXT_TICKER_NAME, TRADING_TICKER_NAME):
#         # STEP 1: FETCH THE CANDLE DATA
#         ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
#         decimals_quantity = check_decimals(trading_ticker_name, binance_client)
        
#         if ticker_data is not None:
#             # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
#             trade_rec_type = get_trade_recommendation(ticker_data, RSI_OVERSOLD, RSI_OVERBOUGHT)
#             print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} {ccxt_ticker_name} TRADING RECOMMENDATION: {trade_rec_type}')

#             # STEP 3: EXECUTE THE BUY OPERATION
#             if (trade_rec_type == 'BUY' and not currently_holding):
#                 print(f'Placing {trade_rec_type} order')
#                 trade_successful, holding_quantity, order_price = execute_trade(binance_client, trade_rec_type, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
#                 currently_holding = not currently_holding if trade_successful else currently_holding
#                 buy_position = True
                
#                 # CHECK SELL OPERATION
#                 while buy_position:
#                     ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
#                     confirm_sell = confirm_sell_operation(ticker_data, 1.1, exchange, ccxt_ticker_name, order_price)
#                     print(confirm_sell)
#                     if (confirm_sell == "SELL"):
#                         print(f'Placing SELL order')
#                         trade_successful, holding_quantity, order_price = execute_trade(binance_client, confirm_sell, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
#                         currently_holding = not currently_holding if trade_successful else currently_holding
#                         buy_position = not buy_position
                    
#                     time.sleep(15)

#         else:
#             print(f'Unable to fetch ticker data - {ccxt_ticker_name}. Retrying!!')
#             time.sleep(5)

#     print("Waiting for next candle...")
#     time.sleep(CANDLE_DURATION_IN_MIN*60)

In [6]:
# CCXT_TICKER_NAME = ['DOT/USDT', 'DOGE/USDT', 'AVAX/USDT', 'MATIC/USDT']
# TRADING_TICKER_NAME = ['DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'MATICUSDT']

# currently_holding = False
# while 1:

#     while datetime.now().minute not in [0, 15, 30, 45]:
#         print("Not in mark yet...")
#         time.sleep(1)

#     for ccxt_ticker_name, trading_ticker_name in zip(CCXT_TICKER_NAME, TRADING_TICKER_NAME):
#         # STEP 1: FETCH THE CANDLE DATA
#         ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
#         decimals_quantity = check_decimals(trading_ticker_name, binance_client)
        
#         if ticker_data is not None:
#             # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
#             trade_rec_type = get_trade_recommendation(ticker_data, RSI_OVERSOLD, RSI_OVERBOUGHT)
#             print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} {ccxt_ticker_name} TRADING RECOMMENDATION: {trade_rec_type}')

#             # STEP 3: EXECUTE THE BUY OPERATION
#             if (trade_rec_type == 'BUY' and not currently_holding):
#                 print(f'Placing {trade_rec_type} order')
#                 trade_successful, holding_quantity, order_price = execute_trade(binance_client, trade_rec_type, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
#                 currently_holding = not currently_holding if trade_successful else currently_holding
#                 buy_position = True
                
#                 # CHECK SELL OPERATION
#                 while buy_position:
#                     ticker_data = fetch_data(exchange, ccxt_ticker_name, CANDLE_DURATION_IN_MIN)
#                     confirm_sell = confirm_sell_operation(ticker_data, 1, exchange, ccxt_ticker_name, order_price)
#                     print(confirm_sell)
#                     if (confirm_sell == "SELL"):
#                         print(f'Placing SELL order')
#                         trade_successful, holding_quantity, order_price = execute_trade(binance_client, confirm_sell, trading_ticker_name, INVESTMENT_AMOUNT_DOLLARS, decimals_quantity, holding_quantity)
#                         currently_holding = not currently_holding if trade_successful else currently_holding
#                         buy_position = not buy_position
                    
#                     time.sleep(15)
                
#                 break
                
#         else:
#             print(f'Unable to fetch ticker data - {ccxt_ticker_name}. Retrying!!')
#             time.sleep(5)
    
#     print("Waiting for next candle...")
#     time.sleep(CANDLE_DURATION_IN_MIN*60)